<a href="https://colab.research.google.com/github/trchudley/GEOG2462/blob/main/Short_Scripts/Week_1_Download_Single_Scene_Landsat_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Downloading a single scene from Google Earth Engine


This script provides an additional opportunity to use Landsat 5, **at your own risk** (it should be perfectly possible to succeed in this assessment using the 10 years of Landsat 8 you have access to!). Landsat 5 was launched in 1984, and the TM sensor failed in 2012. As Landsat 6 did not make orbit, and Landsat 7 developed a scan line corrector issue which made data difficult to use, Landsat 5 was kept going for so long that it holds the title of the longest-operating Earth-observing satellite mission in history.

The Thematic Mapper (TM) sensing instrument on board Landsat 5 is not the same Operational Land Imager (OLI) on board Landsat 8. As a result **bands have different radiometric resolutions and number designations**. Note that below, we have visualised the RGB 'true colour' image using bands 3/2/1 for red/green/blue rather than 4/3/2 as we would for Landsat 8. I have also exported the GeoTiff as bands 3/2/1/4/5 rather than 4/3/2/5/6 to keep things consistently ordered as red/green/blue/NIR/SWIR.

> ⚠️
> **To convert your own NDI equations to the appropriate bands, you will have to consult the following tables for the TM and OLI equivalent bands**:
> https://www.usgs.gov/faqs/what-are-band-designations-landsat-satellites




## Logging in to Google Earth Engine

Ensure the project name is your own, created upon registration with GEE. You can easily register one at the [following link](https://code.earthengine.google.com/register) - just make sure to select `Unpaid Usage` > `Academia & Research`).

In [ ]:
import ee
import geemap
import time

ee.Authenticate()  # Trigger the authentication flow.
ee.Initialize(project='ee-trchudley')    # Change to your own default project name.

## Define editable variables

This is the only cell you will need to edit in this notebook.

In [ ]:

# Define search parameters
latitude = 54.77   # Degrees of latitude
longitude = -1.58  # Degrees of longitude
size = 10000  # Size of AOI, in metres
region_name = 'durham'  # AOI name, for filename construction

# Define search range, within which the least cloudy image will be found
date_start = '1994-05-01'
date_end = '1994-09-30'

# Google Drive export folder
folder = 'scires_project_2C'


# Search for data

In [ ]:
# Get search region geometry
point = ee.Geometry.Point(longitude, latitude)  # Create a point
region = point.buffer(size/2).bounds()  # Buffer the point to a 2D shape

# Get Landsat 5 image collection
landsat5_collection = ee.ImageCollection("LANDSAT/LT05/C02/T1_TOA")

# Filter to desired region and date bounds
landsat5_collection = landsat5_collection.filterBounds(region)
landsat5_collection = landsat5_collection.filterDate(date_start, date_end)

# landsat8_collection

# Get least cloudy image and clip to search region
image = landsat5_collection.sort('CLOUD_COVER').first()
image = image.clip(region)

image

Sanity check image:

In [ ]:
Map = geemap.Map()  # Create empty map

max_reflectance = 0.25 # Set the upper limit of reflectance to visualise.
                       # Play with this value (between 0-1) to see what it
                       # does. It will need to be higher for snowy/icy
                       # scenes

visParams = {'bands': ['B3', 'B2', 'B1'], 'max': max_reflectance}
Map.addLayer(image, visParams, 'Colour Composite Image')

Map.centerObject(region, zoom=12)
Map

# Download the image

In [ ]:
# Get the data of the image from the metadata
date_string = image.get('DATE_ACQUIRED').getInfo()

# Now we will construct the filename automatically
filename = region_name + '_' + date_string + '_image'

# Visualise for testing
print("The image will be saved to your Google Drive at:\n" + folder + '/' + filename + '.tif')

# Export the image, specifying scale and region.
task = ee.batch.Export.image.toDrive(**{
    'image': image.select(['B3', 'B2', 'B1', 'B4', 'B5']),
    'description': filename,
    'folder': folder,
    'scale': 30,
    'region': region.getInfo()['coordinates']
})
task.start()

while task.active():
  print('Task processing ongoing... (id: {}).'.format(task.id))
  time.sleep(5)

print('Finished processing. Image is exported to your Drive.')